# Classifying Fashion-MNIST

이제 neural network을 만들고 훈련시킬 차례이다. MNIST dataset에 대해서 drop-in 대체로 [Fashion-MNIST dataset](https://github.com/zalandoresearch/fashion-mnist)를 사용할 예정이다. MNIST는 실제로 97% 정확도 이상을 쉽게 달성할 수 있다. Fashion-MNIST는 28x28 greyscale 옷 images의 집합이다. MNIST보다 더 복잡해서 network의 실제 성능과 실제 세상에서 사용하는 dataset의을 더잘 나타낸다.

<img src='assets/fashion-mnist-sprite.png' width=500px>

이 notebook에서는 여러분 자신의 neural network를 만들어보자. 대부분은 Part 3에서 그냥 복사 및 붙여넣기하면 된다. 코드를 스스로 작성해서 동작시키는 것이 중요하다. 이를 통해서 이전에 notebook을 참고해도 된다.

먼저 리소스를 import하고 Fashion-MNIST dataset을 `tensorflow_datasets`에서 다운받자.

## Import Resources

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import numpy as np
import matplotlib.pyplot as plt

import tensorflow as tf
import tensorflow_datasets as tfds
tfds.disable_progress_bar()

In [ ]:
import logging
logger = tf.get_logger()
logger.setLevel(logging.ERROR)

In [ ]:
print('Using:')
print('\t\u2022 TensorFlow version:', tf.__version__)
print('\t\u2022 tf.keras version:', tf.keras.__version__)
print('\t\u2022 Running on GPU' if tf.test.is_gpu_available() else '\t\u2022 GPU device not found. Running on CPU')

## Load the Dataset

이전에 했던 것처럼 `tensorflow_datasets`을 사용해서 Fashion-MNIST dataset을 load할 예정이다. 하지만 이 경우 `split` argument를 생략할 것이다. 이 말은 `tensorflow_datasets`가 `split`를 위해서 기본값인 `split=None`를 사용할 것이다. `split=None`일 때, `tensorflow_datasets`은 로딩하는 dataset에 대해서 유효한 모든 splits를 가지는 **dictionary**를 반환한다. 하지만 split이 명시적으로 `split='train'`와 같이 주어진 경우라면 `tensorflow_datasets`은 `tf.data.Dataset` object를 반환한다.

이 경우 `fashion_mnist` dataset를 load할 것이다. 만약 [documentation](https://www.tensorflow.org/datasets/catalog/fashion_mnist#statistics)를 보면 이 특수한 dataset은 2개 splits를 가지는 것을 보게 되며 이름은 `train`와 `test`이다. `train` split은 60,000 예제를 가지고 있고 `test` split은 10,000 예제를 가지고 있다.

이제 `fashion_mnist` dataset를 load하고 반환 값을 조사하자.

In [ ]:
dataset, dataset_info = tfds.load('fashion_mnist', as_supervised = True, with_info = True)

In [ ]:
# Check that dataset is a dictionary
print('dataset has type:', type(dataset))

# Print the keys of the dataset dictionary
print('\nThe keys of dataset are:', list(dataset.keys()))

아래 cell에서 training data를 저장하고 test data를 다른 변수에 저장한다.

In [ ]:
training_set, test_set = dataset['train'], dataset['test']

이제 `dataset_info`를 살펴보자.

In [ ]:
# Display the dataset_info
dataset_info

매우 쉽게 `dataset_info` 정보에 접근할 수 있다. 보는 바와 같이 `features` 와 `splits` info는 dictionaries에 포함된다. 이 dictionaries에 특정 key와 value에 접근해서 원하는 정보에 접근할 수 있다. 이 dictionaries에 있는 특정 key의 값을 살펴보자.:

In [ ]:
dataset_info.features['image']

In [ ]:
dataset_info.features['label']

In [ ]:
dataset_info.splits['train']

원하는 정보에 접근하기 위해서 dot 표기법을 사용할 수 있다. 아래 몇가지 예제가 있다. :

In [ ]:
shape_images = dataset_info.features['image'].shape
num_classes = dataset_info.features['label'].num_classes

num_training_examples  = dataset_info.splits['train'].num_examples
num_test_examples = dataset_info.splits['test'].num_examples

print('There are {:,} classes in our dataset'.format(num_classes))
print('The images in our dataset have shape:', shape_images)

print('\nThere are {:,} images in the test set'.format(num_test_examples))
print('There are {:,} images in the training set'.format(num_training_examples))

## Explore the Dataset

이 dataset내에 images은 28 $\times$ 28 arrays이고 `[0, 255]` 범위의 pixel 값을 가진다. *labels*은 정수의 array이고 `[0, 9]` 범위에 있다. 이는 image가 나타내는 clothing의 *class*와 일치하다.:

<table>
  <tr>
    <th>Label</th>
    <th>Class</th> 
  </tr>
  <tr>
    <td>0</td>
    <td>T-shirt/top</td> 
  </tr>
  <tr>
    <td>1</td>
    <td>Trouser</td> 
  </tr>
    <tr>
    <td>2</td>
    <td>Pullover</td> 
  </tr>
    <tr>
    <td>3</td>
    <td>Dress</td> 
  </tr>
    <tr>
    <td>4</td>
    <td>Coat</td> 
  </tr>
    <tr>
    <td>5</td>
    <td>Sandal</td> 
  </tr>
    <tr>
    <td>6</td>
    <td>Shirt</td> 
  </tr>
    <tr>
    <td>7</td>
    <td>Sneaker</td> 
  </tr>
    <tr>
    <td>8</td>
    <td>Bag</td> 
  </tr>
    <tr>
    <td>9</td>
    <td>Ankle boot</td> 
  </tr>
</table>

각 image는 single label에 매핑된다. *class names*은 dataset에 포함되지 않기 때문에 image를 그릴때 사용해서 생성한다.:

In [ ]:
class_names = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat', 
               'Sandal',      'Shirt',   'Sneaker',  'Bag',   'Ankle boot']

In [ ]:
for image, label in training_set.take(1):
    print('The images in the training set have:\n\u2022 dtype:', image.dtype, '\n\u2022 shape:', image.shape)

In [ ]:
for image, label in training_set.take(1):
    image = image.numpy().squeeze()
    label = label.numpy()

plt.imshow(image, cmap= plt.cm.binary)
plt.colorbar()
plt.show()

print('The label of this image is:', label)
print('The class name of this image is:', class_names[label])

## Create Pipeline

In [ ]:
def normalize(image, label):
    image = tf.cast(image, tf.float32)
    image /= 255
    return image, label

batch_size = 64

training_batches = training_set.cache().shuffle(num_training_examples//4).batch(batch_size).map(normalize).prefetch(1)
testing_batches = test_set.cache().batch(batch_size).map(normalize).prefetch(1)

## Build the Model

> **연습문제:** 여기서 여러분의 neural network을 정의해야한다. 자유롭게 원하는 만큼의 layer와 neurons를 사용해서 model을 생성하자. MNIST로 각 image는 28 $\times$ 28 이고 전체 784 pixel이 되고 10개 classes가 있다. model에는 적어도 한개 이상의 hidden layer를 포함해야만 한다. hidden layers를 위한 ReLU activation과 output layer를 위한 softmax activation function 사용하도록 하자.

In [ ]:
## Solution


## Train the Model

> **Exercise:** Compile the model you created above using an `adam` optimizer, a `sparse_categorical_crossentropy` loss function, and the `accuracy` metric. Then train the model for 5 epochs. You should be able to get the training loss below 0.4.

In [ ]:
## Solution


## Evaluate Loss and Accuracy on the Test Set

Now let's see how the model performs on the test set. This time, we will use all the examples in our test set to assess the loss and accuracy of our model. Remember, the images in the test are images the model has never seen before.

In [ ]:
loss, accuracy = my_model.evaluate(testing_batches)

print('\nLoss on the TEST Set: {:,.3f}'.format(loss))
print('Accuracy on the TEST Set: {:.3%}'.format(accuracy))

## Check Predictions

In [ ]:
for image_batch, label_batch in testing_batches.take(1):
    ps = my_model.predict(image_batch)
    first_image = image_batch.numpy().squeeze()[0]
    first_label = label_batch.numpy()[0]

fig, (ax1, ax2) = plt.subplots(figsize=(6,9), ncols=2)
ax1.imshow(first_image, cmap = plt.cm.binary)
ax1.axis('off')
ax1.set_title(class_names[first_label])
ax2.barh(np.arange(10), ps[0])
ax2.set_aspect(0.1)
ax2.set_yticks(np.arange(10))
ax2.set_yticklabels(class_names, size='small');
ax2.set_title('Class Probability')
ax2.set_xlim(0, 1.1)
plt.tight_layout()